In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import squarify # pip install squarify
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# suprimimos la notacion cientifica en los outputs
pd.options.display.float_format = '{:20,.2f}'.format

labelsDF = pd.read_csv('train_labels.csv')
valuesDF = pd.read_csv('train_values.csv')
test_values = pd.read_csv('test_values.csv', index_col='building_id')

# Asignacion de datos a columnas segun enunciado

In [5]:
valuesProcessed = valuesDF
valuesProcessed["land_surface_condition"]               = valuesProcessed["land_surface_condition"].astype("category")
valuesProcessed["foundation_type"]                      = valuesProcessed["foundation_type"].astype("category")
valuesProcessed["roof_type"]                            = valuesProcessed["roof_type"].astype("category")
valuesProcessed["ground_floor_type"]                    = valuesProcessed["ground_floor_type"].astype("category")
valuesProcessed["other_floor_type"]                     = valuesProcessed["other_floor_type"].astype("category")
valuesProcessed["position"]                             = valuesProcessed["position"].astype("category")
valuesProcessed["plan_configuration"]                   = valuesProcessed["plan_configuration"].astype("category")
valuesProcessed["legal_ownership_status"]               = valuesProcessed["legal_ownership_status"].astype("category")

test_values["land_surface_condition"]               = test_values["land_surface_condition"].astype("category")
test_values["foundation_type"]                      = test_values["foundation_type"].astype("category")
test_values["roof_type"]                            = test_values["roof_type"].astype("category")
test_values["ground_floor_type"]                    = test_values["ground_floor_type"].astype("category")
test_values["other_floor_type"]                     = test_values["other_floor_type"].astype("category")
test_values["position"]                             = test_values["position"].astype("category")
test_values["plan_configuration"]                   = test_values["plan_configuration"].astype("category")
test_values["legal_ownership_status"]               = test_values["legal_ownership_status"].astype("category")

In [6]:
valuesJoined = valuesProcessed.merge(labelsDF, how='inner', on='building_id')
valuesJoined

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,0,0,0,2
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,0,0,0,3
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,0,0,0,2


# ¿Tiene más familias que el promedio?

In [7]:
valuesJoined.loc[valuesJoined['count_families'] > 0].describe()['count_families']

count             239,739.00
mean                    1.07
std                     0.31
min                     1.00
25%                     1.00
50%                     1.00
75%                     1.00
max                     9.00
Name: count_families, dtype: float64

In [8]:
valuesJoined['more_families_than_mean'] = valuesJoined.apply(lambda x: x['count_families'] > 1.07, axis=1)
test_values['more_families_than_mean'] = test_values.apply(lambda x: x['count_families'] > 1.07, axis=1)

# ¿Es más antiguo que el promedio?

In [9]:
valuesJoined.describe()['age']

count             260,601.00
mean                   26.54
std                    73.57
min                     0.00
25%                    10.00
50%                    15.00
75%                    30.00
max                   995.00
Name: age, dtype: float64

In [10]:
valuesJoined['older_than_mean'] = valuesJoined.apply(lambda x: x['age'] > 26.54, axis=1)
test_values['older_than_mean'] = test_values.apply(lambda x: x['age'] > 26.55, axis=1)

# Cantidad de materiales usados

In [11]:
subset = valuesJoined[['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
                       'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
                       'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick',
                       'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
                       'has_superstructure_rc_engineered', 'has_superstructure_other']]
valuesJoined["cant_materials"] = subset.sum(axis=1)

subset_test = test_values[['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
                       'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
                       'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick',
                       'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
                       'has_superstructure_rc_engineered', 'has_superstructure_other']]
test_values["cant_materials"] = subset_test.sum(axis=1)

# Daño promedio por cantidad de pisos

In [12]:
def mean_damage_per_floor_cant(x):
    if x == 1:
        return 2.04
    elif x == 2:
        return 2.24
    elif x == 3:
        return 2.4
    elif x == 4:
        return 2.12
    elif x == 5:
        return 2.11
    elif x == 6:
        return 1.86
    elif x == 7:
        return 1.85
    elif x == 8:
        return 2
    elif x == 9:
        return 1
    else:
        return 0

In [13]:
valuesJoined['mean_damage_per_count_floor'] = valuesJoined.apply(lambda x: mean_damage_per_floor_cant(x['count_floors_pre_eq']), axis=1)
test_values['mean_damage_per_count_floor'] = test_values.apply(lambda x: mean_damage_per_floor_cant(x['count_floors_pre_eq']), axis=1)

In [14]:
# Viendo esto podemos agregar otra columna que asigne 1 si tiene la cantidad pisos que resultó con daño considerable
def bad_cant_floor(x):
    if (x > 0 and x < 6) or x == 8:
        return 1
    else:
        return 0
valuesJoined['bad_cant_floor'] = valuesJoined.apply(lambda x: bad_cant_floor(x['count_floors_pre_eq']), axis=1)
test_values['bad_cant_floor'] = test_values.apply(lambda x: bad_cant_floor(x['count_floors_pre_eq']), axis=1)

# Daño promedio por geo level 1

In [15]:
mean_damage_level_1 = valuesJoined[['geo_level_1_id', 'damage_grade']].groupby('geo_level_1_id').mean()

In [16]:
mean_damage_level_1.columns = ['mean_damage_level_1']

In [17]:
test_values.reset_index(inplace = True)

In [18]:
valuesJoined = valuesJoined.merge(mean_damage_level_1, how = 'inner', on = 'geo_level_1_id')
test_values = test_values.merge(mean_damage_level_1, how = 'inner', on = 'geo_level_1_id')

# Buscamos relacion entre altura y area
unidades de altura por cada unidad de area

In [19]:
valuesJoined['height per area'] = valuesJoined.height_percentage / valuesJoined.area_percentage
test_values['height per area'] = test_values.height_percentage / test_values.area_percentage

# Promedio de daño por plan configuration en cada geo level 1
Vimos en el TP1 que el diseño sismico con el que fue construido era influyente en el daño que recibia dicho edificio. Si lo combinamos con la ubicacion geografica del mismo puede darnos buenos resultados.

In [20]:
mean_grouped_subset = valuesJoined[['geo_level_1_id', 'plan_configuration', 'damage_grade']].groupby(['geo_level_1_id', 'plan_configuration']).mean()

In [21]:
mean_grouped_subset.columns = ['mean_damage_level_1_per_plan_configuration']
mean_grouped_subset

mean_damage_level_1_per_plan_configuration
geo_level_1_id plan_configuration                                            
0              a                                                         2.12
               c                                                         2.00
               d                                                         2.06
               f                                                          nan
               m                                                         2.40
...                                                                       ...
30             n                                                          nan
               o                                                         2.00
               q                                                         1.85
               s                                                         2.00
               u                                                         1.93

[310 rows x 1 columns]

In [22]:
valuesJoined = valuesJoined.merge(mean_grouped_subset, how = 'inner', on = ['geo_level_1_id', 'plan_configuration'])
test_values = test_values.merge(mean_grouped_subset, how = 'inner', on = ['geo_level_1_id', 'plan_configuration'])

# ¿Pertenece a la región más dañada?
En el TP1 vimos que la region 17-363-8236, siendo geo level 1 id, 2 y 3 respectivamente, fue la región con mayor cantidad de edificios con damage_grade = 3. Haremos una columna con un valor representativo que implicaria si esta o no en los distintos niveles de dicha region.

In [23]:
def value_of_region(x,l1,l2,l3):
    if x['geo_level_1_id'] == l1:
        if x['geo_level_2_id'] == l2:
            if x['geo_level_3_id'] == l3:
                return 3
            else:
                return 2
        else:
            return 1
    else:
        return 0        

In [24]:
valuesJoined['most_damaged_region'] = valuesJoined.apply(lambda x: value_of_region(x,17,363,8236), axis=1)
test_values['most_damaged_region'] = test_values.apply(lambda x: value_of_region(x,17,363,8236), axis=1)

# ¿Pertenece a la región menos dañada?
Asi como con la región más dañada, hicimos el mismo análisis para la región con la mayor cantidad de edificios con damage_grade = 1. La misma es 26-39-9133

In [25]:
valuesJoined['less_damaged_region'] = valuesJoined.apply(lambda x: value_of_region(x,26,39,9133), axis=1)
test_values['less_damaged_region'] = test_values.apply(lambda x: value_of_region(x,26,39,9133), axis=1)

# ¿Tiene la combinación de diseño sísimico y materiales más dañados?
Como pudimos ver en el TP1, se vio una diferencia considerable del promedio de daño que sufrieron los edificios dependiendo de los materiales con los cuales habían sido construidos. Notamos que el diseño sísmico que resultó "peor" en cuanto a la resistencia contra la catástofre fue el denomidado por "n". Y dentro del mismo, los peores resultados lo obtuvieron aquellas edificaciones que habían sido contstruidas con:
- Adobe/Barro
- Piedra
- Barro-Ladrillos
- Otro

Es por esto que haremos un feature en el cual se indique si la misma tuvo, en la construcción el plan sísmico mencionado y alguno de los 4 materiales listados.

In [26]:
def has_the_most_damaged_combination(x):
    if x['plan_configuration'] == 'n':
        if x['has_superstructure_adobe_mud'] or x['has_superstructure_stone_flag'] or x['has_superstructure_mud_mortar_brick'] or x['has_superstructure_other']:
            return 1
    return 0

In [27]:
valuesJoined['has_most_damaged_construct_combination'] = valuesJoined.\
apply(lambda x: has_the_most_damaged_combination(x), axis=1)

test_values['has_most_damaged_construct_combination'] = test_values.\
apply(lambda x: has_the_most_damaged_combination(x), axis=1)

# ¿Tiene el material más usado?
Los materiales con los que fueron construidos las edificaciones son un factor que influye en el daño que tuvieron los mismos. Veremos cuál es el material que se utilizaba en la mayoría de las edificaciones y haremos un feature que indique si la edificación posee o no el mismo. El hecho de que sean muchos los edificios que lo tienen puede significar que es un material eficiente en cuanto a calidad/costo.

In [28]:
most_used_train_material = valuesJoined[['has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
              'has_superstructure_cement_mortar_stone',
              'has_superstructure_mud_mortar_brick',
              'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
              'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
              'has_superstructure_rc_engineered', 'has_superstructure_other']].sum().sort_values(ascending=False).keys()[0]

most_used_test_material = valuesJoined[['has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
              'has_superstructure_cement_mortar_stone',
              'has_superstructure_mud_mortar_brick',
              'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
              'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
              'has_superstructure_rc_engineered', 'has_superstructure_other']].sum().sort_values(ascending=False).keys()[0]

valuesJoined['has_most_used_material'] = valuesJoined.apply(lambda x: x[most_used_train_material], axis=1)
test_values['has_most_used_material'] = test_values.apply(lambda x: x[most_used_test_material], axis=1)

# ¿Tiene el material menos usado?
Por lo mencionado anteriormente, también buscaremos si la edificación fue construida con el material que fue utilizado en la menor cantidad de edificios. Este feature nos dará un poco más de información ya que la variabilidad en el material más usado puede ser muy grande, creemos que en este caso podría ser menor y darnos más información para el modelo.

In [29]:
least_used_train_material = valuesJoined[['has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
              'has_superstructure_cement_mortar_stone',
              'has_superstructure_mud_mortar_brick',
              'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
              'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
              'has_superstructure_rc_engineered', 'has_superstructure_other']].sum().sort_values().keys()[0]

least_used_test_material = valuesJoined[['has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
              'has_superstructure_cement_mortar_stone',
              'has_superstructure_mud_mortar_brick',
              'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
              'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
              'has_superstructure_rc_engineered', 'has_superstructure_other']].sum().sort_values().keys()[0]

valuesJoined['has_least_used_material'] = valuesJoined.apply(lambda x: x[least_used_train_material], axis=1)
test_values['has_least_used_material'] = test_values.apply(lambda x: x[least_used_test_material], axis=1)

# ¿Tiene el material mas/menos resistente?
Como ya dijimos, los materiales con los que fueron construidas las edificaciones es influyente. Por esto es que, ademas de cuales fueron los más y menos utilizados, investigaremos el promedio de daño. Y veremos si cada uno de los edificios tiene el material con mayor y menor promedio.

In [30]:
materials = ['has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
              'has_superstructure_cement_mortar_stone',
              'has_superstructure_mud_mortar_brick',
              'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
              'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
              'has_superstructure_rc_engineered', 'has_superstructure_other']

higher_mean = 0
lower_mean = 3
for mat in materials:
    col_mean = valuesJoined.loc[valuesJoined[mat] == 1, ['damage_grade']].mean().values[0]
    if col_mean >= higher_mean:
        higher_mean = col_mean
        higher_mean_material = mat
    if col_mean <= lower_mean:
        lower_mean = col_mean
        lower_mean_material = mat

In [31]:
valuesJoined['has_higher_mean_material'] = valuesJoined.apply(lambda x: x[higher_mean_material], axis=1)
valuesJoined['has_lower_mean_material'] = valuesJoined.apply(lambda x: x[lower_mean_material], axis=1)

test_values['has_higher_mean_material'] = test_values.apply(lambda x: x[higher_mean_material], axis=1)
test_values['has_lower_mean_material'] = test_values.apply(lambda x: x[lower_mean_material], axis=1)

# Daño promedio por geo level 2 y 3
Si bien este es un dato que solo va a servir para este sismo, es decir para la competencia y no para predecir otros terremotos, vimos que el daño promedio por geo level 1 arrojó buenos resultados y es un feature muy importante. Por lo tanto haremos lo mismo con las regiones más específicas para ver si logramos seguir mejorando el modelo.

In [32]:
valuesJoined

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,mean_damage_level_1,height per area,mean_damage_level_1_per_plan_configuration,most_damaged_region,less_damaged_region,has_most_damaged_construct_combination,has_most_used_material,has_least_used_material,has_higher_mean_material,has_lower_mean_material
0,802906,6,487,12198,2,30,6,5,t,r,...,2.16,0.83,2.17,0,0,0,1,0,0,0
1,633759,6,706,12267,2,5,7,5,t,r,...,2.16,0.71,2.17,0,0,0,1,0,0,0
2,157370,6,706,11722,2,20,4,4,t,r,...,2.16,1.00,2.17,0,0,0,1,0,0,0
3,33651,6,1210,9423,3,40,5,7,t,r,...,2.16,1.40,2.17,0,0,0,1,0,0,0
4,1005736,6,724,7981,2,10,5,5,t,r,...,2.16,1.00,2.17,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,665334,29,903,10668,1,20,9,3,n,r,...,2.08,0.33,1.80,0,0,0,1,0,0,0
260597,506593,29,903,10668,3,55,7,7,n,r,...,2.08,1.00,1.80,0,0,0,1,1,0,0
260598,994870,29,903,10668,1,30,10,4,n,r,...,2.08,0.40,1.80,0,0,0,1,0,0,0
260599,345899,29,903,10668,2,25,15,6,t,r,...,2.08,0.40,1.80,0,0,0,1,0,0,0


In [33]:
mean_damage_level_2 = valuesJoined[['geo_level_2_id', 'damage_grade']].groupby('geo_level_2_id').mean()
mean_damage_level_2.columns = ['mean_damage_level_2']

valuesJoined = valuesJoined.merge(mean_damage_level_2, how = 'left', on = 'geo_level_2_id')
test_values = test_values.merge(mean_damage_level_2, how = 'left', on = 'geo_level_2_id')

In [34]:
mean_damage_level_3 = valuesJoined[['geo_level_3_id', 'damage_grade']].groupby('geo_level_3_id').mean()
mean_damage_level_3.columns = ['mean_damage_level_3']

valuesJoined = valuesJoined.merge(mean_damage_level_3, how = 'left', on = 'geo_level_3_id')
test_values = test_values.merge(mean_damage_level_3, how = 'left', on = 'geo_level_3_id')

In [35]:
val = valuesJoined['geo_level_2_id'].drop_duplicates().sort_values().array

In [36]:
tes = test_values['geo_level_2_id'].drop_duplicates().sort_values().array

In [37]:
for i in range (1428):
    if i in val:
        if i in tes:
            continue
        else:
            print('esta en val y no en tes')
            print(i)
    else:
        if i in tes:
            print('ESTA EN TES Y NO EN VAL')
            print(i)

esta en val y no en tes
15
esta en val y no en tes
78
esta en val y no en tes
100
esta en val y no en tes
186
esta en val y no en tes
201
esta en val y no en tes
231
ESTA EN TES Y NO EN VAL
235
ESTA EN TES Y NO EN VAL
295
esta en val y no en tes
301
esta en val y no en tes
319
esta en val y no en tes
325
esta en val y no en tes
361
esta en val y no en tes
372
esta en val y no en tes
392
esta en val y no en tes
441
ESTA EN TES Y NO EN VAL
504
esta en val y no en tes
505
esta en val y no en tes
515
esta en val y no en tes
524
ESTA EN TES Y NO EN VAL
538
esta en val y no en tes
552
esta en val y no en tes
627
esta en val y no en tes
633
esta en val y no en tes
637
esta en val y no en tes
665
esta en val y no en tes
686
esta en val y no en tes
711
esta en val y no en tes
739
esta en val y no en tes
758
esta en val y no en tes
771
esta en val y no en tes
786
esta en val y no en tes
833
esta en val y no en tes
862
esta en val y no en tes
865
esta en val y no en tes
894
esta en val y no en te

### Aqui vemos que hay algunos valores de geo level 2 y 3 que estan en el set de test y no en el de train. Por lo tanto, al hacer el merge, tendremos valores en nulo para estos casos. Para evitar esto, pondremos un 0 como valor. El 0 es un valor que no esta en ninguno de los casos, entonces evitariamos que el modelo los relacione con aquellos que si tienen un valor promedio, y relacionaria entre ellos a los que tienen valor cero. Esto podria no estar tan bueno, pero con el geo level 1 obtendremos la diferencia entre ellos.

### En el caso de plan configuration tenemso un problema parecido. No existe la combinacion, en el set de train, del geo level 1 con ese diseño sismico, y si en el set de test. Haciendo referencia a que no tenemos datos de esos casos, pondremos un cero de valor.

In [38]:
for col in test_values.columns:
    if test_values[col].isnull().values.any():
        print(col)
        print(test_values[col].isnull().sum())

mean_damage_level_1_per_plan_configuration
9
mean_damage_level_2
5
mean_damage_level_3
327


In [39]:
test_values['mean_damage_level_1_per_plan_configuration'].fillna(value=0, inplace=True)
test_values['mean_damage_level_2'].fillna(value=0, inplace=True)
test_values['mean_damage_level_3'].fillna(value=0, inplace=True)
test_values.isnull().values.any()

False

# Uso de la edificación

De la manera en que estan indicados los usos de los edificios puede generar relaciones que son incorrectas, por ejemplo que una casa y un departamento de policia son iguales por no ser hoteles. Para evitar esto, y generar una especie de diferenciación, vamos a crear una columna única en la cual se asigne un valor numérico por cada uso.

In [40]:
def use(x):
    if not x['has_secondary_use']: # Es una casa
        return 1
    if x['has_secondary_use_agriculture']: # Es de agricultura
        return 2
    if x['has_secondary_use_hotel']: # Es un hotel
        return 3
    if x['has_secondary_use_rental']: # Es de alquiler
        return 4
    if x['has_secondary_use_institution']: # Es una institucion
        return 5
    if x['has_secondary_use_school']: # Es una escuela
        return 6
    if x['has_secondary_use_industry']: # Es una industria
        return 7
    if x['has_secondary_use_health_post']: # Es un puesto de salud
        return 8
    if x['has_secondary_use_gov_office']: # Es una oficina de gobierno
        return 9
    if x['has_secondary_use_use_police']: # Es una estacion de policias
        return 10
    if x['has_secondary_use_other']: # tiene otro uso
        return 11

In [41]:
valuesJoined['use'] = valuesJoined.apply(lambda x: use(x), axis=1)
test_values['use'] = test_values.apply(lambda x: use(x), axis=1)

# De los elementos categoricos que definen un edificio, veamos cuales tienen mayor variacion y por lo tanto son los que mas podrian definir su durabilidad. 




In [42]:
values = ['count_floors_pre_eq', 'age',  'area_percentage', 'height_percentage', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type',
         'other_floor_type', 'position', 'plan_configuration']
filteredValues = valuesJoined[["building_id"] + values + ["damage_grade"]]

for elem in values:
    print(filteredValues[[elem, "damage_grade"]].groupby(elem).agg({"count","mean"}))

                    damage_grade                     
                           count                 mean
count_floors_pre_eq                                  
1                          40441                 2.04
2                         156623                 2.24
3                          55617                 2.40
4                           5424                 2.12
5                           2246                 2.11
6                            209                 1.86
7                             39                 1.85
8                              1                 2.00
9                              1                 1.00
    damage_grade                     
           count                 mean
age                                  
0          26041                 1.96
5          33697                 2.10
10         38896                 2.20
15         36010                 2.26
20         32182                 2.30
25         24366                 2.33
30        

de aca podemos ver que los mas influyentes son 
foundation_type, roof_type, ground_floor_type, other_floor_type
Para cada uno de estos, puedo hacer una columna que vea si tiene el mejor, o el peor. 

por ejemplo, foundation_type tiene 2 que funcionan relativamente mal y otro 3 que funcionan bien. los agrupare de esa forma. 

roof type, lo mismo 2 malos 1 mejor. 

ground floor, el mejor es V y los otros son similares.

para other_floor_type. 1 bueno y 3 relativamente malos y similares.  

In [43]:



valuesJoined['has_good_foundation_type'] = valuesJoined.apply(lambda x: 1 if x["foundation_type"] in ["i","u","w"] else 0, axis=1)
valuesJoined['has_good_roof_type'] = valuesJoined.apply(lambda x: 1 if x["roof_type"] == "x" else 0, axis=1)
valuesJoined['has_good_ground_floor_type'] = valuesJoined.apply(lambda x: 1 if x["ground_floor_type"] == "v" else 0, axis=1)
valuesJoined['has_good_other_floor_type'] = valuesJoined.apply(lambda x: 1 if x["other_floor_type"] == "s" else 0, axis=1)

test_values['has_good_foundation_type'] = test_values.apply(lambda x: 1 if x["foundation_type"] in ["i","u","w"] else 0, axis=1)
test_values['has_good_roof_type'] = test_values.apply(lambda x: 1 if x["roof_type"] == "x" else 0, axis=1)
test_values['has_good_ground_floor_type'] = test_values.apply(lambda x: 1 if x["ground_floor_type"] == "v" else 0, axis=1)
test_values['has_good_other_floor_type'] = test_values.apply(lambda x: 1 if x["other_floor_type"] == "s" else 0, axis=1)


# Descargamos los csv
Esta celda siempre va al final

In [44]:
#valuesJoined.drop('damage_grade', axis=1, inplace=True)
valuesJoined.to_csv('train_engineered_features.csv')
test_values.to_csv('test_engineered_features.csv')